# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?



In [1]:
// initialize
var path = require('path');

// TLDR; What do we end up with?
var importNotebook = (notebook) => Promise.resolve({});
var getCells = ((notebook, types = ['code']) => Promise.resolve([]));
var runAllInNEL = (cells) => Promise.resolve({values: [], session: {}});
var assignResults = (results, filename) => results;

// provide a function for importing any notebook as a module and executing it
var importNotebook = (notebook) => {
    var filename = path.resolve(notebook);
    var dirname = path.resolve(path.dirname(filename));
    // only read javascript kernels
    return getCells(filename, ['javascript', 'code'])
    .then(cells => cells.map(c => c.source.join('').trim()))
    .then(cells => ['__filename="'+filename+'";\n',
                   '__dirname="'+dirname+'";\n'].concat(cells))
    .then(cells => runAllInNEL(cells))
    .then((results) => assignResults(results, notebook));
};

// $$.done(); for our dumb parser



What do I need to have installed?

How to sanity check the environment?


In [1]:
// initialize

var execSync = require('child_process').execSync;
var deps = ['npm install', 'JSONStream', 'nel', 'path', 'fs', 'vm'];
try {
    require.resolve('JSONStream');
    require.resolve('nel');
} catch (e) {
    execSync(deps.slice(0, 3).join(' '), {stdio: 'inherit'});
}

// $$.done() for our dumb parser


How to stream json using a {match} function?

In [1]:
// initialize
var JSONStream = require('JSONStream');
(streamJson = (file, parse, match = ((c) => true)) => {
    var file = fs.createReadStream(file)
        .pipe(JSONStream.parse(parse));
    file.on('data', (m) => match(m));
    // create a promise out of this stream
    return new Promise((resolve, reject) => {
        file.on('error', e => reject(e, file));
        file.on('close', () => resolve(file))
    });
});
// $$.done() for our dumb parser



[Function: streamJson]


How to parse cells of {types} from a notebook?



In [34]:
// initialize

// How to use getCells?
(getCells = (notebook, types = ['*', 'code']) => {
    var cells = [];
    var kernel;
    return streamJson(notebook, [true, {emitPath: true}], (match) => {
        if(match.path[0] === 'metadata'
           && match.path[1] === 'kernelspec') {
            kernel = match.value;
        } else if(match.path[0] === 'cells'
                  && types.indexOf(match.value.cell_type) > -1) {
            cells[cells.length] = match.value;
        }
    }).then((file) => {
        if (types.indexOf(kernel.language) === -1
           && types.indexOf('*') === -1) {
            console.log('tried to import ' + types + JSON.stringify(kernel))
            return [];
        }
        return cells;
    });
});

// $$.done() for our dumb parser


[Function: getCells]

How to run all promises sequentially?



In [1]:
// initialize

var runAllPromises = (promises) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (typeof func == 'function'
                ? (new Promise(func)) : func)
                .then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([]));
};
(runAllPromises);
// $$.done() for our dumb parser



How to run all code cells in jupyter?

In [ ]:
// initialize
var nel = require('nel');
// How to create VM promises out of the cell content?
var sess;
(runAllInNEL = (cells, isNewSession) => {
    if(typeof sess === 'undefined' || isNewSession === true) {
        sess = new nel.Session();
    }
    return runAllPromises(
        cells.map(
            (cell) =>
            (resolve) =>
            sess.execute(cell, {
                onStdout: console.log,
                onStderr: console.error,
                onSuccess: (o) => resolve(o),
                onError: (o) => resolve(o),
                beforeRun: () => {}})
        )).then((r) => (r.session = sess) && (r));
});

// $$.done() for our dumb parser



In [6]:
// initialize
if (typeof imported != 'object' || imported == null) { var imported = {}; }

// display a heading when new notebooks are imported
var logIm = (cells, filename) => {
    return 'Running ' + filename  
                + ' with ' + cells.length + ' cells';
};

function wrapCallback (func, session) {
    // arguments doesn't work on anonymous functions like the return value:
    return (a1,a2,a3,a4,a5,a6,a7,a8,a9) => {
        var exec = '$$.async(); Promise.resolve(' 
        + func + '.apply(this,' 
        + JSON.stringify([a1,a2,a3,a4,a5,a6,a7,a8,a9]) 
        + ')).then(r => $$.mime({"text/json": r}))' 
        + '.catch(r => $$.mime({"text/json": r}));';
        return runAllInNEL([exec], session)
            .then(r => {
                var value = typeof r[0]['mime'] !== 'undefined' 
                    && typeof r[0]['mime']['text/json'] !== 'undefined' 
                    ? r[0]['mime']['text/json']
                    : r[0];
                return value;
        });
    };
}

// cache results when notebooks are processed
(assignResults = (results, notebook) => {
    var filename = path.basename(notebook);
    var session = results['session'];
    var results = results.reduce((obj, r, i) => {
        if (typeof r === 'object' && typeof r['mime'] !== 'undefined'
            && typeof r['mime']['text/plain'] !== 'undefined') {
            var re = (/\[Function:\s*(.*?)\]/ig);
            var name = re.exec(r['mime']['text/plain']);
            if(name !== null) {
                obj[filename+'['+i+']'] = wrapCallback(name[1], session);
                obj[name[1]] = obj[filename+'['+i+']'];
            }
        }
        return obj;
    }, {});
    Object.assign(global, results);
    imported[filename] = results;
    return results;
});

// $$.done() for our dumb parser


How to import a {notebook}?

How to test the import feature?


In [7]:

// initialize
if (typeof imported != 'object' || imported == null) { var imported = {}; }
// How to test if a notebook has already been imported?
var oldImport = importNotebook;
(importNotebook = (notebook) => {
    var filename = path.basename(notebook);
    if(typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    if (typeof imported[filename] !== 'undefined') {
        console.log('Already imported');
        console.log(Object.keys(imported));
        return Promise.resolve(imported[filename]);
    }
    return oldImport(notebook);
});

// $$.done() for our dumb parser


In [2]:
// initialize

if (typeof imported != 'object' || imported == null) { var imported = {}; }

imported['How to import a notebook.ipynb'] = {
    'How to find answers.ipynb[2]': streamJson,
    streamJson: streamJson,
    'How to find answers.ipynb[3]': getCells,
    getCells: getCells,
    'How to find answers.ipynb[4]': runAllPromises,
    runAllPromises: runAllPromises,
    'How to find answers.ipynb[5]': runAllInNEL,
    runAllInNEL: runAllInNEL,
    'How to find answers.ipynb[6]': assignResults,
    assignResults: assignResults,
    'How to find answers.ipynb[7]': importNotebook,
    importNotebook: importNotebook,
    'How to find answers.ipynb[8]': imported,
    imported: imported
};
(imported);

// $$.done() for our dumb parser



[eval]


ReferenceError: streamJson is not defined

These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?

